In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("Tickets.csv")

In [3]:
df.head()

,Sno,QUESTION,SOLUTION
0,1,Issue with Power bi date slicer,If you are experiencing an issue with the date...
1,2,What are some common issues that can arise wit...,If you are experiencing an issue with the date...
2,3,How do I resolve issues with the date slicer i...,If you are experiencing an issue with the date...
3,4,I'm having trouble getting the date slicer in ...,If you are experiencing an issue with the date...
4,5,What could be causing the date slicer in Power...,If you are experiencing an issue with the date...


In [4]:
df['Sno'] = df['Sno'].astype("str").apply(lambda x : "C"+x)

In [5]:
df.head()

,Sno,QUESTION,SOLUTION
0,C1,Issue with Power bi date slicer,If you are experiencing an issue with the date...
1,C2,What are some common issues that can arise wit...,If you are experiencing an issue with the date...
2,C3,How do I resolve issues with the date slicer i...,If you are experiencing an issue with the date...
3,C4,I'm having trouble getting the date slicer in ...,If you are experiencing an issue with the date...
4,C5,What could be causing the date slicer in Power...,If you are experiencing an issue with the date...


In [6]:
dict_data = df.set_index('Sno')['QUESTION'].to_dict()

In [7]:
import json
with open("ticketsdata.json",'w') as f:
    json.dump(dict_data,f,indent=4)

In [8]:
dict_data_Ans = df.set_index("Sno")['SOLUTION'].to_dict()

In [9]:
import json
with open("ticketsdatasoultions.json",'w') as f:
    json.dump(dict_data_Ans,f,indent=4)

In [10]:
import google.generativeai as genai
import os
from dotenv import load_dotenv
import ast
_ = load_dotenv()
genai.configure(api_key=os.environ['GOOGLE_API_KEY'])

c:\Users\rakesh.nadiminti\Desktop\Hackathon\RiskLens\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
# Create the model
# generation_config = {
#     "temperature": 1,
#     "top_p": 0.95,
#     "top_k": 64,
#     "max_output_tokens": 8124,
#     "response_mime_type": "text/plain",
# }

model = genai.GenerativeModel(
    model_name="gemini-1.5-flash",
    # generation_config=generation_config,
)

# Query = "Issue with Power bi date slicer"

# Query = """I'm having trouble getting the date slicer in Power BI to work properly. Are 
# there any specific settings or configurations I should check to resolve the issue?"""
Query = """What are the common causes of Power BI API token issue in the Azure \nADFS setup, and how can they be resolved?"""
prompt = """Given data is caseid and query in key value format 
Go through the given below Data:  """ + json.dumps(dict_data) +"""Provide only top 5 CaseID in the Response For Given Query in this below format ["C1","C2","C3","C4","C5"] """ + Query
response = model.generate_content(prompt)
CaseIds = list(set(ast.literal_eval(response.text)))
print(CaseIds)

['C89', 'C88', 'C86', 'C90', 'C87']


In [12]:
print(type(CaseIds))

<class 'list'>


In [13]:
# with open("ticketsdatasoultions.json",'r') as f:
#     data = json.load(f)
# data[response.text.strip('\n').strip()]

In [14]:
%pip install openpyxl


[notice] A new release of pip is available: 23.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
New_df = pd.read_excel('AzureGCPServiceOfferingsTicketsFinal.xlsx') 

In [16]:
def gemini_risk_identifier(row):
    
    model = genai.GenerativeModel(
    model_name="gemini-1.5-flash",
    )
    prompt = "Given Ticket Description:" + str(row[2]) + "Ticket severity:" + str(row[4]) + "Ticket prority:" + str(row[5]) + "Resolution Time:" + str(row[10]) + """\n\n Based on the given data Find 3 Categories of the Risks that a software ticket should 
      in the below format only risk1,risk2,risk3. \n from the below list of categories""" + """["Customer Dissatisfaction","Lost Productivity","Financial Losses","Security Vulnerabilities","Operational Disruptions","Legal and Compliance Issues","Increased Support Costs","Negative Impact on Employee Morale","Missed Business Opportunities","Damaged Brand Reputation"]
      Please provide the response with only categories separated by comma and dont provide detail description"""
    response = model.generate_content(prompt)
    print(response.text)
    return response.text

In [17]:
import random
import numpy as np
categories = ["Customer Dissatisfaction","Lost Productivity","Financial Losses","Security Vulnerabilities","Operational Disruptions","Legal and Compliance Issues","Increased Support Costs","Negative Impact on Employee Morale","Missed Business Opportunities","Damaged Brand Reputation"]
random_values = [np.random.choice(categories, size=3) for _ in range(len(New_df))]
New_df["RISKS"] = [", ".join(values) for values in random_values]

In [18]:
New_df.head(2)

,Sn No,TECHNOLOGY,QUESTION,SOLUTION,SEVERITY,PRIORITY,CREATED_USER,CREATED_DATE,CLOSED_USER,CLOSED_DATE,RESOLUTION_TIME(HOURS),CITY,LATITUDE,LONGITUDE,RISKS
0,1,PowerBI,Issue with Power bi date slicer,If you are experiencing an issue with the date...,medium,medium,David,2022-11-08,Lucas,2022-11-09,3,Elmhurst,40.747185,-117.135437,"Customer Dissatisfaction, Missed Business Oppo..."
1,2,PowerBI,The date slicer is not filtering data correctly,If your Power BI date slicer is not filtering ...,critical,medium,David,2022-11-08,Gabriella,2022-11-11,6,San Diego,32.775814,-117.135437,"Increased Support Costs, Missed Business Oppor..."


In [19]:
New_df.to_csv("AzureGCPServiceOfferingsTicketsFinal.csv",index=False)

In [20]:
New_df['Sn No'] = New_df['Sn No'].astype(str).apply(lambda x: "C"+x)

In [21]:
New_df

,Sn No,TECHNOLOGY,QUESTION,SOLUTION,SEVERITY,PRIORITY,CREATED_USER,CREATED_DATE,CLOSED_USER,CLOSED_DATE,RESOLUTION_TIME(HOURS),CITY,LATITUDE,LONGITUDE,RISKS
0,C1,PowerBI,Issue with Power bi date slicer,If you are experiencing an issue with the date...,medium,medium,David,2022-11-08,Lucas,2022-11-09,3,Elmhurst,40.747185,-117.135437,"Customer Dissatisfaction, Missed Business Oppo..."
1,C2,PowerBI,The date slicer is not filtering data correctly,If your Power BI date slicer is not filtering ...,critical,medium,David,2022-11-08,Gabriella,2022-11-11,6,San Diego,32.775814,-117.135437,"Increased Support Costs, Missed Business Oppor..."
2,C3,PowerBI,What are the solutions for resolving issues wi...,Try the following solutions:\n1. Ensure that y...,critical,high,David,2022-06-12,James,2022-06-13,6,Aurora,39.709320,-104.814583,"Missed Business Opportunities, Negative Impact..."
3,C4,PowerBI,Unchanged Azure Dataflow Shows Validation Erro...,1. Check if the query you have used is valid.\...,high,low,David,2021-10-11,Daniel,2021-10-12,12,Victorville,34.509357,-117.330292,"Customer Dissatisfaction, Damaged Brand Reputa..."
4,C5,PowerBI,Power BI Dataflow - Validating Queries - Extre...,There are a few known limitations to using Ent...,medium,medium,Emily,2021-10-11,Madison,2021-10-14,11,Aurora,39.751770,-104.782967,"Missed Business Opportunities, Increased Suppo..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
485,C486,Cloud Scheduler,"Error: ""Permission denied"" or ""Insufficient pe...",Ensure that the user or service account has th...,high,high,Bob,2023-05-28,Madison,2023-05-31,34,San Antonio,29.529984,-98.608292,"Financial Losses, Increased Support Costs, Fin..."
486,C487,Cloud Scheduler,"Error: ""Invalid job configuration"" or ""Failed ...","Double-check the job configuration, including ...",high,medium,Bob,2023-10-26,Katherine,2023-10-29,5,Lawrenceville,33.941853,-84.086632,"Legal and Compliance Issues, Missed Business O..."
487,C488,Cloud Scheduler,"Error: ""Authentication failed"" or ""Invalid cre...",Ensure that the target endpoint or service bei...,low,low,Bob,2023-10-26,Iris,2023-10-27,21,Orlando,28.556259,-81.275871,"Customer Dissatisfaction, Financial Losses, Ne..."
488,C489,Cloud Scheduler,"Error: ""Failed to reach the target endpoint"" o...",Solution:\n1. Check the target endpoint's avai...,high,low,Emily,2022-04-05,Madison,2022-04-07,36,Fort Washington,38.742664,-76.991798,"Operational Disruptions, Damaged Brand Reputat..."


In [26]:
import json
import time
json_data = {}
# Sn No	TECHNOLOGY	QUESTION	SOLUTION	SEVERITY	PRIORITY	CREATED_USER	CREATED_DATE	
# CLOSED_USER	CLOSED_DATE	RESOLUTION_TIME(HOURS)	CITY	LATITUDE	LONGITUDE
call_count = 0
max_requests_per_minute = 10
for k,v in New_df.iterrows():
    # print(type(v))
    # print(v[1])
    if call_count == max_requests_per_minute:
            print(call_count)
            call_count = 0
            time.sleep(60)
    else:
        call_count += 1   
        key = v.iloc[0]
        json_data[key] = [
                {
                    "Technology":v.iloc[1],
                    "Question":v.iloc[2],
                    "Solution":v.iloc[3],
                    "Severity":v.iloc[4],
                    "Priority":v.iloc[5],
                    "CreatedUser":v.iloc[6],
                    "CreatedDate":v.iloc[7].strftime('%Y-%m-%d %H:%M:%S').strip(" 00:00:00"),
                    "Assignee":v.iloc[8],
                    "ClosedDate":v.iloc[9].strftime('%Y-%m-%d %H:%M:%S').strip(" 00:00:00"),
                    "ResolutionTime":v.iloc[10],
                    "City":v.iloc[11],
                    "Latitude":v.iloc[12],
                    "Longitude":v.iloc[13],
                    "Risks":gemini_risk_identifier(v)
                }
        ]

with open("newtickets.json",'w') as f:
    json.dump(json_data,f,indent=4)

C:\Users\rakesh.nadiminti\AppData\Local\Temp\ipykernel_8612\471952005.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  prompt = "Given Ticket Description:" + str(row[2]) + "Ticket severity:" + str(row[4]) + "Ticket prority:" + str(row[5]) + "Resolution Time:" + str(row[10]) + """\n\n Based on the given data Find 3 Categories of the Risks that a software ticket should


Customer Dissatisfaction, Lost Productivity, Increased Support Costs 

Customer Dissatisfaction, Lost Productivity, Operational Disruptions 

Customer Dissatisfaction, Lost Productivity, Damaged Brand Reputation 

Customer Dissatisfaction, Lost Productivity, Increased Support Costs 

Customer Dissatisfaction, Lost Productivity, Increased Support Costs 

Customer Dissatisfaction, Lost Productivity, Increased Support Costs 

Customer Dissatisfaction, Lost Productivity, Increased Support Costs 

Customer Dissatisfaction, Lost Productivity, Increased Support Costs 

Customer Dissatisfaction, Lost Productivity, Increased Support Costs 

Customer Dissatisfaction, Lost Productivity, Increased Support Costs 

10
Customer Dissatisfaction, Lost Productivity, Operational Disruptions 

Customer Dissatisfaction, Lost Productivity, Increased Support Costs 

Customer Dissatisfaction, Lost Productivity, Damaged Brand Reputation 

Customer Dissatisfaction, Lost Productivity, Increased Support Costs 

C

In [ ]:
def gemini_chat_infterface(Query):
    with open("newtickets.json","r") as f:
        data = json.load(f)
    model = genai.GenerativeModel(
    model_name="gemini-1.5-flash",
    )
    prompt = """Given data is caseid and query in key value format 
    Go through the given below Data:  """ + json.dumps(dict_data) +"""Provide only top 5 CaseID in the Response For Given Query in this below format ["C1","C2","C3","C4","C5"] """ + Query
    response = model.generate_content(prompt)
    print(response.text)
    CaseIds = list(set(ast.literal_eval(response.text)))
    # caseId = data[response.text.strip('\n').strip()]
    result = {}
    for case in CaseIds:
        result.update({case:data[case.strip('\n').strip()][0]})            
    return result

In [ ]:
Query = "Issue with Power bi date slicer"
solution = gemini_chat_infterface(Query)

In [ ]:
solution

In [ ]:
prompt = """Generate only the HTML table code  for the below data """+json.dumps(solution) + """Provide the Proper Styling to Table"""
response = model.generate_content(prompt)
# ,tools='code_execution'
print(response.text)
with open("sample.txt","w") as f:
    f.write(response.text)

In [1]:
# %pip install llama_index
# %pip install llama-index-embeddings-gemini
# %pip install llama-index-llms-gemini
# %pip install google-generativeai
# %pip install PyPDF2

from llama_index.core.node_parser import TokenTextSplitter
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import Settings
from llama_index.embeddings.gemini import GeminiEmbedding
from llama_index.llms.gemini import Gemini
from llama_index.core import Document
from llama_index.core import VectorStoreIndex
from llama_index.core import SimpleDirectoryReader
import re
import PyPDF2
from dotenv import load_dotenv, find_dotenv
import os
import google.generativeai as genai
_ = load_dotenv(find_dotenv())
GOOGLE_API_KEY = os.environ['GOOGLE_API_KEY']
genai.configure(api_key=GOOGLE_API_KEY)


def pdf_2_txt(pdf_path):
    try:
        pdf_file = open(pdf_path, 'rb')
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        text_content = ' '
        for page_number in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_number]
            text_content += page.extract_text()
        pdf_file.close()
        text_content = re.sub(r'\s+', ' ', text_content)
    except Exception as e:
        print("Error:", e)
    return text_content


def knowledge_whisperer(user_query):
    llm = Gemini(model_name="models/gemini-1.5-flash-latest")
    embed_model = GeminiEmbedding(model_name="models/embedding-001",
                                  generation_config={"temperature": 0.7, "topP": 0.8, "topK": 40})

    text_splitter = SentenceSplitter(chunk_size=1024, chunk_overlap=20)
    splitter = TokenTextSplitter(chunk_size=1024, chunk_overlap=20)

    # global settings
    Settings.llm = llm
    Settings.embed_model = embed_model
    Settings.text_splitter = text_splitter
    Workspace_name = "knowledgeBase"
    ticket_files = os.listdir(Workspace_name)
    contents = ""
    for file in ticket_files:
        print(file+" Completed")
        contents += pdf_2_txt(Workspace_name+"/"+file)
    documents = documents = [Document(text=contents)]
    index = VectorStoreIndex.from_documents(documents, show_progress=True)
    query_engine = index.as_query_engine()
    response = query_engine.query(user_query)
    return response
    print(response)
